# Benodigde libraries importeren

In [47]:
import pandas as pd
from datetime import datetime
import numpy as np
import pandas.api.types as ptypes


# Bestand inladen en data filteren

In [48]:
# excel bestand maxilia geimporteerd en kolommen gefilterd 

df = pd.read_csv(r'/Users/lisannepeeters/expdata 20190912 1002.csv', sep=None, delimiter=None, engine='python', usecols=['email','Orderdate','OrderID','TurnoverLead', 'BuyinLead', 'LeadPhase','LeadStatus'])
df.head()

OrderID  Orderdate                              email  TurnoverLead  \
0    27654  22-4-2014  janneke.van.wingerden@hotmail.com         840.0   
1    30777   9-4-2015             marloessanders@izit.nl         300.0   
2    32291  30-7-2014              info@ferox-design.com         235.0   
3    32291  30-7-2014              info@ferox-design.com         235.0   
4    33824  27-5-2016              johan@thermservice.be           0.0   

   BuyinLead LeadPhase    LeadStatus  
0        0.0     Order  Gefactureerd  
1        0.0     Order  Gefactureerd  
2        0.0     Order  Gefactureerd  
3        0.0     Order  Gefactureerd  
4        0.0     Order  Gefactureerd

In [49]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 273322 entries, 0 to 273321
Data columns (total 7 columns):
OrderID         273322 non-null int64
Orderdate       131183 non-null object
email           272921 non-null object
TurnoverLead    273322 non-null float64
BuyinLead       273322 non-null float64
LeadPhase       273322 non-null object
LeadStatus      271268 non-null object
dtypes: float64(2), int64(1), object(4)
memory usage: 14.6+ MB


In [50]:
# lege rijen 'NaT' eruit gefilterd 
# Email bevat @ controle

df = df.dropna(axis=0, how = 'any', inplace=False)
df = df[~df['email'].str.contains('@')==False]
df.head()

OrderID  Orderdate                              email  TurnoverLead  \
0    27654  22-4-2014  janneke.van.wingerden@hotmail.com         840.0   
1    30777   9-4-2015             marloessanders@izit.nl         300.0   
2    32291  30-7-2014              info@ferox-design.com         235.0   
3    32291  30-7-2014              info@ferox-design.com         235.0   
4    33824  27-5-2016              johan@thermservice.be           0.0   

   BuyinLead LeadPhase    LeadStatus  
0        0.0     Order  Gefactureerd  
1        0.0     Order  Gefactureerd  
2        0.0     Order  Gefactureerd  
3        0.0     Order  Gefactureerd  
4        0.0     Order  Gefactureerd

In [51]:
df['Orderdate']=pd.to_datetime(df['Orderdate'], format='%d-%m-%Y')


In [52]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 130990 entries, 0 to 273321
Data columns (total 7 columns):
OrderID         130990 non-null int64
Orderdate       130990 non-null datetime64[ns]
email           130990 non-null object
TurnoverLead    130990 non-null float64
BuyinLead       130990 non-null float64
LeadPhase       130990 non-null object
LeadStatus      130990 non-null object
dtypes: datetime64[ns](1), float64(2), int64(1), object(3)
memory usage: 8.0+ MB


In [53]:
df.describe()

OrderID   TurnoverLead      BuyinLead
count  130990.000000  130990.000000  130990.000000
mean   149361.316788     831.791600     501.835442
std     60978.915532    2564.755128    1918.010058
min     27654.000000    -392.000000   -1001.510000
25%     95896.250000     183.000000      44.500000
50%    150672.500000     355.000000     177.300000
75%    203525.000000     765.000000     448.450000
max    252808.000000  174519.500000  141338.840000

In [54]:
# Leadphase = order
# Leadstatus = gefactureerd
df = df[~df['LeadPhase'].str.contains('Order')==False]
df = df[~df['LeadStatus'].str.contains('Gefactureerd')==False]
df.head()

OrderID  Orderdate                              email  TurnoverLead  \
0    27654 2014-04-22  janneke.van.wingerden@hotmail.com         840.0   
1    30777 2015-04-09             marloessanders@izit.nl         300.0   
2    32291 2014-07-30              info@ferox-design.com         235.0   
3    32291 2014-07-30              info@ferox-design.com         235.0   
4    33824 2016-05-27              johan@thermservice.be           0.0   

   BuyinLead LeadPhase    LeadStatus  
0        0.0     Order  Gefactureerd  
1        0.0     Order  Gefactureerd  
2        0.0     Order  Gefactureerd  
3        0.0     Order  Gefactureerd  
4        0.0     Order  Gefactureerd

In [55]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 120174 entries, 0 to 272752
Data columns (total 7 columns):
OrderID         120174 non-null int64
Orderdate       120174 non-null datetime64[ns]
email           120174 non-null object
TurnoverLead    120174 non-null float64
BuyinLead       120174 non-null float64
LeadPhase       120174 non-null object
LeadStatus      120174 non-null object
dtypes: datetime64[ns](1), float64(2), int64(1), object(3)
memory usage: 7.3+ MB


In [56]:
# Check data bevat voldoet aan bepaalde condities
assert ptypes.is_datetime64_any_dtype(df['Orderdate'])
assert all(df[df['email'].str.contains('@')])
assert all(df[df['LeadPhase'].str.contains('Order')])
assert all(df[df['LeadStatus'].str.contains('Gefactureerd')])


# Data grouperen per klant

In [57]:
# group by customer ID
# hoeveel orders zijn er gedaan per customer
# totale uitgaven per customer 

df_clv = df.groupby('email').agg({
                                                        'Orderdate': lambda date: (date.max() - date.min()).days,
                                                        'OrderID': lambda num: len(num), 
                                                        'TurnoverLead': lambda price: price.sum()
                                                    }) 
df_clv.head()

Orderdate  OrderID  \
email                                                                    
-                                                           0        1   
03vichy@gmail.com                                           0        1   
06alexisbrunet@gmail.com                                    0        1   
070paulus070@gmail.com                                      0        1   
1.NLD.MINUSMA.GAO.AHDET.COGP.S1@B-SC1.TTN.MINDE...          0        1   

                                                    TurnoverLead  
email                                                             
-                                                         2727.5  
03vichy@gmail.com                                          259.0  
06alexisbrunet@gmail.com                                   180.0  
070paulus070@gmail.com                                      33.5  
1.NLD.MINUSMA.GAO.AHDET.COGP.S1@B-SC1.TTN.MINDE...        1230.5

In [12]:
# rename kolommen 

df_clv.columns = ['days_customer', 'total_transactions', 'total_spent']
df_clv.head()

days_customer  \
email                                                               
03vichy@gmail.com                                               0   
06alexisbrunet@gmail.com                                        0   
070paulus070@gmail.com                                          0   
1.NLD.MINUSMA.GAO.AHDET.COGP.S1@B-SC1.TTN.MINDE...              0   
1.vorsitzende.gv.muenster@gmail.com                             0   

                                                    total_transactions  \
email                                                                    
03vichy@gmail.com                                                    1   
06alexisbrunet@gmail.com                                             1   
070paulus070@gmail.com                                               1   
1.NLD.MINUSMA.GAO.AHDET.COGP.S1@B-SC1.TTN.MINDE...                   1   
1.vorsitzende.gv.muenster@gmail.com                                  2   

                                                    total_spent  
email                                                            
03vichy@gmail.com                                         259.0  
06alexisbrunet@gmail.com                                  180.0  
070paulus070@gmail.com                                     33.5  
1.NLD.MINUSMA.GAO.AHDET.COGP.S1@B-SC1.TTN.MINDE...       1230.5  
1.vorsitzende.gv.muenster@gmail.com                       689.0

In [13]:
df_clv.describe()

days_customer  total_transactions   total_spent
count   67777.000000        67777.000000  6.777700e+04
mean       76.580684            1.773050  1.422681e+03
std       225.263499            2.678135  1.018020e+04
min         0.000000            1.000000 -7.840000e+02
25%         0.000000            1.000000  1.770000e+02
50%         0.000000            1.000000  3.505000e+02
75%         0.000000            2.000000  8.950000e+02
max      1686.000000          177.000000  1.974618e+06

# Berekenen van de totale CLV

In [14]:
# gemiddelde order waarde 

avg_order_value = sum(df_clv['total_spent']) / sum(df_clv['total_transactions'])
print(avg_order_value)

802.3922652531487


In [15]:
# hoe vaak koopt iemand gemiddeld iets

purchase_frequency = sum(df_clv['total_transactions']) / df_clv.shape[0]
print(purchase_frequency)

1.7730498546704634


In [16]:
# hoe vaak komt iemand terug

repeat_rate = df_clv[df_clv.total_transactions > 1].shape[0] / df_clv.shape[0] 
print(repeat_rate)

0.2944804284639332


In [17]:
# churnrate: hoeveel % doet na 1 aankoop geen bestelling meer

churn_rate = 1 - repeat_rate
print(churn_rate)

0.7055195715360668


In [18]:
# gemiddelde marge  

profit = sum(df['TurnoverLead']) - sum(df['BuyinLead'])
margin = profit / sum(df['TurnoverLead'])
    
print(margin)

0.4000812090561646


In [19]:
# totale clv & clv marge 
# in het blok hierboven is de marge berekend ('margin'), maar je kan ook een getal noteren ipv 'margin'

total_clv = (avg_order_value * purchase_frequency) / churn_rate 
total_clv_margin = total_clv * margin

print(total_clv) # het eerste getal is de totale clv
print(total_clv_margin) # het tweede getal is de clv waarbij rekening is gehouden met de berekende marge

2016.501804759743
806.764480112216


In [20]:
# Bovenstaande cijfers in kolom neerzetten zodat de data in datastudio kan worden gebruikt
set_total_clv = avg_order_value, purchase_frequency, churn_rate, total_clv, total_clv_margin 
df_total_clv = pd.DataFrame(set_total_clv, columns= ['Waarde'], index = ['Gemiddelde orderwaarde' , 'Frequentie aankoop', 'Churn rate' , 'CLV', 'CLV marge'])
df_clv_values=df_total_clv.T.groupby(level=0).agg(lambda x : x.values.tolist()).stack().apply(pd.Series).unstack().sort_index(level=1,axis=1)
df_clv_values.columns=df_clv_values.columns.droplevel(level=0)
df_clv_values.head()

CLV  CLV marge  Churn rate  Frequentie aankoop  \
Waarde  2016.501805  806.76448     0.70552             1.77305   

        Gemiddelde orderwaarde  
Waarde              802.392265

In [21]:
df_clv_values.to_excel('CLV_waardes.xlsx')

   # Transacties en churn rate per maand uiteenzetten

In [22]:
# transacties per klant per maand uiteenzetten
df_monthly = df.set_index('Orderdate')
df_monthly = df_monthly.pivot_table(
    index=['email'],
    columns=pd.Grouper(freq='M'),
    values='OrderID', 
    aggfunc='count',
    fill_value=0
)

# Format column headers to human readable format
df_monthly.columns = [x.strftime('%b-%Y') for x in df_monthly.columns]

df_monthly

Apr-2014  Jul-2014  \
email                                                                    
03vichy@gmail.com                                          0         0   
06alexisbrunet@gmail.com                                   0         0   
070paulus070@gmail.com                                     0         0   
1.NLD.MINUSMA.GAO.AHDET.COGP.S1@B-SC1.TTN.MINDE...         0         0   
1.vorsitzende.gv.muenster@gmail.com                        0         0   
1.vs@ogv-asperg.de                                         0         0   
111856@student.drenthecollege.nl                           0         0   
1187@nl.mcd.com                                            0         0   
1206admin@nl.mcd.com                                       0         0   
121584@wolfert.nl                                          0         0   
125055@wolfert.nl                                          0         0   
126869@dr.nassaucollege.nl                                 0         0   
126871@dr.nassaucollege.nl                                 0         0   
13erna13@gmail.com                                         0         0   
142189@buas.nl                                             0         0   
17di10@gmail.com                                           0         0   
18rog8d61@gmail.com                                        0         0   
1913stickershop@gmail.com                                  0         0   
19verena94@web.de                                          0         0   
1NLDDMAN.LD.LDGB.MDW_03@dman-sc1.ttn.mindef.nl             0         0   
1Scherrer@gmx.net                                          0         0   
2-dogs@gmx.de                                              0         0   
2.Vorstand@mc-kirchheim.de                                 0         0   
2.vorsitzender@herder-freunde.de                           0         0   
2.vorsitzender@tri-sport.de                                0         0   
200@hollandtechnogroep.nl                                  0         0   
24hdestan2018@gmail.com                                    0         0   
2608.mo@gmail.com                                          0         0   
2anke@gmx.net                                              0         0   
2stformation@gmail.com                                     0         0   
...                                                      ...       ...   
zsteinert@smartPS.de                                       0         0   
zstorni@human-insight.com                                  0         0   
ztaa100@hotmail.com                                        0         0   
zubayr@alice-dsl.de                                        0         0   
zuivel@groenhouten.nl                                      0         0   
zulal.dogan@summumwoman.com                                0         0   
zuleikha@zulutions.nl                                      0         0   
zumsteg@physiowerk19.ch                                    0         0   
zur-nina@gmx.de                                            0         0   
zuster055@gmail.com                                        0         0   
zutphenvan.e@gmail.com                                     0         0   
zvo.zaz@gmail.com                                          0         0   
zw.adlerbande@yahoo.de                                     0         0   
zwaan@home2home.nl                                         0         0   
zwaanj@corderius.nl                                        0         0   
zwanikken@esset.nl                                         0         0   
zweedijk@bowlingcentrum.nl                                 0         0   
zweifel_roman@hotmail.com                                  0         0   
zwemles.breda@clubpellikaan.nl                             0         0   
zwemschool-de-zwemfan@hotmail.com                          0         0   
zwemschooldeoceaan@gmail.com                               0         0   
zwemschoolfranka@gmail.com                              

In [23]:
repeat_rate = df_monthly[df_monthly > 1].count() / df_monthly[df_monthly > 0].count()
df_churn = repeat_rate.to_frame(name='repeat_rate')

In [24]:
df_churn['churn_rate'] = 1 - df_churn['repeat_rate']
df_churn


repeat_rate  churn_rate
Apr-2014     0.000000    1.000000
Jul-2014     1.000000    0.000000
Sep-2014     0.000000    1.000000
Oct-2014     0.333333    0.666667
Nov-2014     0.222222    0.777778
Dec-2014     0.157895    0.842105
Jan-2015     0.113941    0.886059
Feb-2015     0.125000    0.875000
Mar-2015     0.142680    0.857320
Apr-2015     0.146650    0.853350
May-2015     0.140230    0.859770
Jun-2015     0.142574    0.857426
Jul-2015     0.180247    0.819753
Aug-2015     0.176000    0.824000
Sep-2015     0.171210    0.828790
Oct-2015     0.147541    0.852459
Nov-2015     0.131424    0.868576
Dec-2015     0.179775    0.820225
Jan-2016     0.172869    0.827131
Feb-2016     0.156355    0.843645
Mar-2016     0.165217    0.834783
Apr-2016     0.167572    0.832428
May-2016     0.163947    0.836053
Jun-2016     0.160954    0.839046
Jul-2016     0.155031    0.844969
Aug-2016     0.164191    0.835809
Sep-2016     0.158333    0.841667
Oct-2016     0.166092    0.833908
Nov-2016     0.178127    0.821873
Dec-2016     0.196370    0.803630
...               ...         ...
Apr-2017     0.185047    0.814953
May-2017     0.169968    0.830032
Jun-2017     0.165556    0.834444
Jul-2017     0.183575    0.816425
Aug-2017     0.190835    0.809165
Sep-2017     0.197899    0.802101
Oct-2017     0.177732    0.822268
Nov-2017     0.180972    0.819028
Dec-2017     0.167474    0.832526
Jan-2018     0.175289    0.824711
Feb-2018     0.172394    0.827606
Mar-2018     0.173025    0.826975
Apr-2018     0.185223    0.814777
May-2018     0.180513    0.819487
Jun-2018     0.174220    0.825780
Jul-2018     0.181722    0.818278
Aug-2018     0.171990    0.828010
Sep-2018     0.188023    0.811977
Oct-2018     0.193277    0.806723
Nov-2018     0.175806    0.824194
Dec-2018     0.185224    0.814776
Jan-2019     0.222332    0.777668
Feb-2019     0.198260    0.801740
Mar-2019     0.217682    0.782318
Apr-2019     0.225266    0.774734
May-2019     0.241921    0.758079
Jun-2019     0.244318    0.755682
Jul-2019     0.208130    0.791870
Aug-2019     0.184505    0.815495
Sep-2019     0.106236    0.893764

[63 rows x 2 columns]

# CLV per klant uitrekenen

In [25]:
# clv per customer

df_clv['customer_order_value'] = df_clv['total_spent'] / df_clv['total_transactions']
df_clv['customer_clv'] = (df_clv['customer_order_value'] * purchase_frequency) / churn_rate
df_clv['customer_clv_margin'] = df_clv['customer_clv'] * margin

df_clv.head()

days_customer  \
email                                                               
03vichy@gmail.com                                               0   
06alexisbrunet@gmail.com                                        0   
070paulus070@gmail.com                                          0   
1.NLD.MINUSMA.GAO.AHDET.COGP.S1@B-SC1.TTN.MINDE...              0   
1.vorsitzende.gv.muenster@gmail.com                             0   

                                                    total_transactions  \
email                                                                    
03vichy@gmail.com                                                    1   
06alexisbrunet@gmail.com                                             1   
070paulus070@gmail.com                                               1   
1.NLD.MINUSMA.GAO.AHDET.COGP.S1@B-SC1.TTN.MINDE...                   1   
1.vorsitzende.gv.muenster@gmail.com                                  2   

                                                    total_spent  \
email                                                             
03vichy@gmail.com                                         259.0   
06alexisbrunet@gmail.com                                  180.0   
070paulus070@gmail.com                                     33.5   
1.NLD.MINUSMA.GAO.AHDET.COGP.S1@B-SC1.TTN.MINDE...       1230.5   
1.vorsitzende.gv.muenster@gmail.com                       689.0   

                                                    customer_order_value  \
email                                                                      
03vichy@gmail.com                                                  259.0   
06alexisbrunet@gmail.com                                           180.0   
070paulus070@gmail.com                                              33.5   
1.NLD.MINUSMA.GAO.AHDET.COGP.S1@B-SC1.TTN.MINDE...                1230.5   
1.vorsitzende.gv.muenster@gmail.com                                344.5   

                                                    customer_clv  \
email                                                              
03vichy@gmail.com                                     650.896064   
06alexisbrunet@gmail.com                              452.360199   
070paulus070@gmail.com                                 84.189259   
1.NLD.MINUSMA.GAO.AHDET.COGP.S1@B-SC1.TTN.MINDE...   3092.384583   
1.vorsitzende.gv.muenster@gmail.com                   865.767159   

                                                    customer_clv_margin  
email                                                                    
03vichy@gmail.com                                            260.411284  
06alexisbrunet@gmail.com                                     180.980815  
070paulus070@gmail.com                                        33.682541  
1.NLD.MINUSMA.GAO.AHDET.COGP.S1@B-SC1.TTN.MINDE...          1237.204963  
1.vorsitzende.gv.muenster@gmail.com                          346.377172

In [26]:
df_clv.describe()

days_customer  total_transactions   total_spent  customer_order_value  \
count   67777.000000        67777.000000  6.777700e+04          67777.000000   
mean       76.580684            1.773050  1.422681e+03            548.221568   
std       225.263499            2.678135  1.018020e+04           1049.502799   
min         0.000000            1.000000 -7.840000e+02           -392.000000   
25%         0.000000            1.000000  1.770000e+02            163.000000   
50%         0.000000            1.000000  3.505000e+02            290.700000   
75%         0.000000            2.000000  8.950000e+02            574.560000   
max      1686.000000          177.000000  1.974618e+06          61600.000000   

        customer_clv  customer_clv_margin  
count   67777.000000         67777.000000  
mean     1377.742319           551.208813  
std      2637.518307          1055.221513  
min      -985.139989          -394.135998  
25%       409.637291           163.888183  
50%       730.561722           292.284017  
75%      1443.933755           577.690763  
max    154807.712577         61935.656819

In [27]:
df_clv.to_excel('CLV_per_klant.xlsx')

In [28]:
# Uiteenzetting uitgaves klanten per maand
df['month_yr'] = df['Orderdate'].apply(lambda x: x.strftime('%b-%Y'))
df.head()

OrderID  Orderdate                              email  TurnoverLead  \
0    27654 2014-04-22  janneke.van.wingerden@hotmail.com         840.0   
1    30777 2015-04-09             marloessanders@izit.nl         300.0   
2    32291 2014-07-30              info@ferox-design.com         235.0   
3    32291 2014-07-30              info@ferox-design.com         235.0   
4    33824 2016-05-27              johan@thermservice.be           0.0   

   BuyinLead LeadPhase    LeadStatus  month_yr  
0        0.0     Order  Gefactureerd  Apr-2014  
1        0.0     Order  Gefactureerd  Apr-2015  
2        0.0     Order  Gefactureerd  Jul-2014  
3        0.0     Order  Gefactureerd  Jul-2014  
4        0.0     Order  Gefactureerd  May-2016

In [29]:
#totale transacties / opbrengst per maand

df_transacties = df.groupby('month_yr').aggregate({
                                                        'OrderID': lambda num: len(num), 
                                                        'TurnoverLead': lambda price: price.sum(),
                                                        'email': pd.Series.nunique
                                                    }) 
df_transacties.head()

OrderID  TurnoverLead  email
month_yr                              
Apr-2014        1        840.00      1
Apr-2015      964     560079.82    791
Apr-2016     1394    1058527.53   1104
Apr-2017     1394    1092140.07   1070
Apr-2018     2577    2085363.70   1949

In [30]:
df_transacties.columns = ['total_transactions', 'total_spent', 'total_unique_customers']
df_transacties.head()

total_transactions  total_spent  total_unique_customers
month_yr                                                         
Apr-2014                   1       840.00                       1
Apr-2015                 964    560079.82                     791
Apr-2016                1394   1058527.53                    1104
Apr-2017                1394   1092140.07                    1070
Apr-2018                2577   2085363.70                    1949

In [31]:
df_transacties.describe()

total_transactions   total_spent  total_unique_customers
count           63.000000  6.300000e+01               63.000000
mean          1907.492063  1.530557e+06             1459.507937
std           1125.410318  9.606221e+05              819.342909
min              1.000000  2.190000e+02                1.000000
25%           1164.000000  8.268239e+05              931.500000
50%           1694.000000  1.357479e+06             1348.000000
75%           2623.500000  2.272373e+06             1995.500000
max           4629.000000  3.839136e+06             3538.000000

In [32]:
df_transacties.to_excel('Overzicht_per_maand.xlsx')

# CLV per maand berekenen

In [33]:
# CLV per maand berekenen
df_transacties['purchase_frequency_month'] = df_transacties['total_transactions'] / df_transacties['total_unique_customers']
df_transacties['avg_order_value_month'] = df_transacties['total_spent'] / df_transacties['total_transactions'] 
df_transacties['CLV'] = (df_transacties['avg_order_value_month'] * df_transacties['purchase_frequency_month']) / df_churn['churn_rate']
df_transacties['CLV_margin'] = df_transacties['CLV']*margin
df_transacties.head()

total_transactions  total_spent  total_unique_customers  \
month_yr                                                            
Apr-2014                   1       840.00                       1   
Apr-2015                 964    560079.82                     791   
Apr-2016                1394   1058527.53                    1104   
Apr-2017                1394   1092140.07                    1070   
Apr-2018                2577   2085363.70                    1949   

          purchase_frequency_month  avg_order_value_month          CLV  \
month_yr                                                                 
Apr-2014                  1.000000             840.000000   840.000000   
Apr-2015                  1.218710             580.995664   829.747881   
Apr-2016                  1.262681             759.345430  1151.825386   
Apr-2017                  1.302804             783.457726  1252.454209   
Apr-2018                  1.322217             809.221459  1313.201322   

          CLV_margin  
month_yr              
Apr-2014  336.068216  
Apr-2015  331.966536  
Apr-2016  460.823693  
Apr-2017  501.083394  
Apr-2018  525.387173

# Statistieken CLV per klant

In [34]:
# statistieken van clv per customer 

import statistics

df_clv_list = df_clv['customer_clv'].values

x1 = statistics.mean(df_clv_list) # eerste getal is gemiddelde
x2 = statistics.median(df_clv_list) # tweede getal is mediaan
x3 = statistics.mode(df_clv_list) # derde getal is de modus 
x4 = statistics.stdev(df_clv_list) # vierde getal is de standaarddeviatie 

print(x1) 
print(x2) 
print(x3) 
print(x4)

1377.7423190737757
730.5617215274582
0.0
2637.518306649236


In [35]:
# box plot: visuele weergaven van de verdeling van de clv 
# zichtbaar = minimum, maximum, mediaan en kwartielafstanden 

import matplotlib.pyplot as plt 
df_clv.plot(y = 'customer_clv', kind = 'box') 
plt.show()

<Figure size 640x480 with 1 Axes>

In [36]:
# histogram plot: visuele weergaven van frequentie verdeling van de clv 
# hoe vaak komt een clv waarde voor 

import matplotlib.pyplot as plt 
df_clv.plot(y = 'customer_clv', kind = 'hist') 
plt.show()